In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive/FYP")

In [0]:
! pip install hottbox

     |████████████████████████████████| 112kB 40.1MB/s 


In [0]:
import numpy as np
import copy
from hottbox.core import Tensor, unfold, TensorCPD, khatri_rao
from hottbox.algorithms.decomposition import CPD
import scipy as sp

class HRSTM:
  
  def __init__(self, C = 10, max_iter = 100, R = 3):
    self.shape = None
    self.order = None
    self.C = C
    self.R = R
    self.max_iter = max_iter
    
    self.weight = None
    self.b = 0 
    self.iteration = 0
    self.orig_lb = None
    
    self.w_history = []
    
  def fit(self, images, labels):
    
    self.order = images[0].order
    self.shape = images[0].shape
    self.orig_lb = list(set(labels))
    
    binary_labels = [1 if x == self.orig_lb[0] else -1 for x in labels]
    W = self.init_weight()
    U = W.fmat
    core = np.arange(self.R)   
    for i in range(self.max_iter):
      print(i)
      for j in range(self.order):
        U_j = self.cal_U_j(U,j)
        A = np.dot(U_j.T, U_j)
        A_half = sp.linalg.sqrtm(A)
        U_miu = np.dot(U[j], A_half)
        Xj = self.cal_Xj(images, j)
        Xmiu = self.cal_Xmiu(Xj, U_j, A_half)
        
        vec_U, b = self.cal_weight(Xmiu, binary_labels, self.C)
        u_ = vec_U.reshape(self.shape[j], self.R)
        u = np.dot(u_, np.linalg.inv(A_half))
        
        u = u / np.linalg.norm(u)
        #normalization 
        
        U[j] = u
        

      W = TensorCPD(fmat = U, core_values = core)
      self.weight = W
      self.w_history.append(W)
      self.b = b
      self.iteration = i
      if self.converged(): break
        
  
  
  def predict(self, images):
    
    w = self.weight.reconstruct()
    W = np.array(w.data)
    b = self.b
    predict = []
    dist = []
    for im in images:
      tmp = np.array(im.data)
      tmp1 = tmp.copy()
      Wmat = unfold(W,0)
      Wvec = Wmat.reshape(-1,1)
      y = unfold(tmp1,0)
      Y = y.reshape(-1,1)
      
      d = np.dot(Y.T, Wvec) + b
      p = np.sign(d)
      dist.append(d)
      predict.append(p)
    
    return predict, dist
    
  
  def init_weight(self):
    U = []
    for dim in self.shape:
      U.append(np.random.randn(dim * self.R).reshape(dim, self.R))
    
    core = np.arange(self.R)
    
    w_cpd = TensorCPD(fmat = U, core_values = core)
    
    return w_cpd
  
  def cal_U_j(self, U, j):
    U_j = khatri_rao(U, skip_matrix = j)
    
    return U_j
  
  def cal_Xj(self, images, j):
    Xj = []
    for im in images:
      tmp = im.data.copy()
      Xj.append(unfold(tmp, j))
    
    return Xj
  
  def cal_Xmiu(self, Xj, U_j, A_half):
    Xmiu = []
    
    for x in Xj:
      tmp = x.copy()
      tmp2 = np.dot(x,U_j)
      tmp3 = np.dot(tmp2, 1/A_half).reshape(-1,1)
      Xmiu.append(tmp3.squeeze())
    Xmiu = np.array(Xmiu).squeeze()
    return (Xmiu)
  
  def cal_weight(self, Xmiu, binary_labels, C):
    
    alphas, b = self.optimizer(Xmiu, binary_labels, C)
    w = np.dot(alphas.transpose(), Xmiu).squeeze()
    
    return w, b
  
  def optimizer(self, Xmiu, binary_labels, C):
    M = Xmiu.shape[0]
    gamma = 2 * C
    y_train = np.expand_dims(np.array(binary_labels), axis = 1)
    
    omega = np.dot(Xmiu, Xmiu.transpose())
    
    left_column = np.expand_dims(np.append(np.array([0]), np.ones(M)), axis=1)
    right_block = np.append(np.expand_dims(np.ones(M), axis=0),  omega + (1/gamma) * np.eye(M), axis=0)
    params = np.append( left_column, right_block, axis=1)
    RHS = np.append(np.array([[0]]), y_train, axis=0)
    result = np.dot(np.linalg.inv(params), RHS)
    
    b = result[0][0]
    alphas = result[1:, :]
    
    return alphas, b
  
  def converged(self):
    if len(self.w_history) > 5:
      wt1 = self.w_history[-1].reconstruct().data
      wt2 = self.w_history[-2].reconstruct().data
      err = np.linalg.norm(wt1 - wt2)/np.linalg.norm(wt2)
      
      if np.abs(err) < 1e-8:
        return True
      
    return False

In [0]:
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt

def load_file(filename):
  with open (filename,'rb') as fo:
    data = pickle.load(fo, encoding = 'latin1')
    label = data['labels']
    images = data['data']
    Label = np.array(label)
    Images = images.reshape(10000,3,32,32).transpose(0,2,3,1).astype('float')
  return Label, Images

def load_cifar10(root):
  label_temp = []
  image_temp = []
  for i in range (1,6):
    trainfile = os.path.join(root, 'data_batch_%d' %(i,))
    l,i = load_file(trainfile)
    label_temp.append(l)
    image_temp.append(i)
  label_train = np.concatenate(label_temp)
  image_train = np.concatenate(image_temp)
  testfile = os.path.join(root, 'test_batch')
  label_test, image_test = load_file(testfile)
  
  return label_train, label_test, image_train, image_test

def showimage(label, image):
  classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  num_class = len(classes)
  num_sample = 7
  for order, class_name in enumerate(classes):
    index = np.flatnonzero(label == order)
    idxs = np.random.choice(index, num_sample, replace = False, p = None)
    for i, idx in enumerate(idxs):
      plt_idx = i * num_class + order + 1
      plt.subplot(num_sample, num_class, plt_idx)
      plt.imshow(image[idx].astype('uint8'))
      plt.axis('off')
      if i == 0:
        plt.title(class_name)
  plt.show()
  
def class_extract(images, label, c, num):
    idx1 = np.flatnonzero(label == c[0])
    idx1_new = np.random.choice(idx1, num, replace = False, p = None)
    idx2 = np.flatnonzero(label == c[1])
    idx2_new = np.random.choice(idx2, num, replace = False, p = None)
    num = idx1_new.shape[0] + idx2_new.shape[0]
    idx = np.hstack((idx1_new,idx2_new))
    idxs = np.random.choice(idx, num, replace = False, p = None)
    class_image = images[idxs]
    class_label = label[idxs]
    
    return class_image, class_label


def tensor_train(images):
    tensor_im = []
    for i in range(images.shape[0]):
        tem = np.squeeze(images[i])
        tensor_im.append(Tensor(tem))
    
    return tensor_im

label_train, label_test, image_train, image_test = load_cifar10('.')
train_im, train_lb = class_extract(image_train, label_train, [0,2], 2000)
avg_im = np.sum(train_im, axis = 0)/train_im.shape[0]
test_im, test_lb = class_extract(image_test, label_test, [0,2], 1000)
train_im = train_im - avg_im
test_im = test_im - avg_im
print(train_im.shape)
print(train_lb.shape)
Image_train = tensor_train(train_im)
Image_test = tensor_train(test_im)
Label_train = np.ones(train_lb.shape)
Label_test = np.ones(test_lb.shape)
Label_train[train_lb == 2] = -1
Label_test[test_lb == 2]= -1

(4000, 32, 32, 3)
(4000,)


In [0]:
stm = HRSTM(C=10, max_iter=20, R=30)
stm.fit(Image_train, Label_train)

0
1
2
3


/usr/local/lib/python3.6/dist-packages/hottbox/core/operations.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
  result[:, i] = temp  # the i-t column is the kronecker product of all the i-th columns of all matrices:


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [0]:
y_tmp, dd = stm.predict(Image_test)
#print(y_tmp)
result = np.zeros(len(Label_test))
succ = 0
for i in range(len(Label_test)):
    #result[i], dec = stm.predict(Image_test[i])
    if Label_test[i] == y_tmp[i]:
       succ = succ + 1
rate = succ/len(Label_test)
print(rate)

0.3035


In [0]:
tt = np.array(Image_test[0].data)
t1 = unfold(tt,0)